In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

### Sequential Flow

In [3]:
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

In [4]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    system_message="You are a creative writer. Please write a story based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on the text generated by the writer.",
    model_client=model_client,
    system_message="You are a reviewer. Please provide feedback on the text generated by the writer.",
)

In [5]:
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

graph = builder.build()

In [6]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [7]:
team =GraphFlow([writer,reviewer], graph)

In [8]:
stream = team.run_stream(task="write a poem about australia in 30 words."
                         )
async for event in stream:
    print(event)

id='5d2452a5-7d66-4bc7-ac7d-60da8f20bf01' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 9, 15, 9, 11, 34, 654043, tzinfo=datetime.timezone.utc) content='write a poem about australia in 30 words.' type='TextMessage'
id='c7f002a2-ed83-49f4-a69f-c57fe4022821' source='Writer' models_usage=RequestUsage(prompt_tokens=38, completion_tokens=49) metadata={} created_at=datetime.datetime(2025, 9, 15, 9, 11, 36, 676043, tzinfo=datetime.timezone.utc) content="Sunlit shores and deserts wide,  \nKangaroos and emus glide.  \nEucalyptus whispers in the breeze,  \nGreat Barrier's coral seas.  \nFrom Outback red to coastal blue,  \nAustralia’s spirit shines through.  " type='TextMessage'
id='acf59e26-961c-43b6-865c-2f63a460fb72' source='Reviewer' models_usage=RequestUsage(prompt_tokens=93, completion_tokens=150) metadata={} created_at=datetime.datetime(2025, 9, 15, 9, 11, 39, 313606, tzinfo=datetime.timezone.utc) content='Your poem captures the essence of Australia beauti

In [15]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient



    # Initialize agents with OpenAI model clients.
model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
agent_a = AssistantAgent("A", model_client=model_client, system_message="You are a helpful assistant.")
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to Chinese.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Japanese.")
agent_d = AssistantAgent("D", model_client=model_client, system_message="Summarize.")


# Create a directed graph with fan-out flow A -> (B, C).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c).add_node(agent_d)
builder.add_edge(agent_a, agent_b).add_edge(agent_a, agent_c)
builder.add_edge(agent_b, agent_d).add_edge(agent_c, agent_d)
graph = builder.build()

# Create a GraphFlow team with the directed graph.
team = GraphFlow(
    participants=[agent_a, agent_b, agent_c, agent_d],
    graph=graph,
    termination_condition=MaxMessageTermination(5),
)

# Run the team and print the events.
async for event in team.run_stream(task="Write a short story about a cat."):
    print(event)

id='947dd1a8-8586-414a-a471-72b0a691ac93' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 9, 15, 9, 16, 10, 975997, tzinfo=datetime.timezone.utc) content='Write a short story about a cat.' type='TextMessage'
id='a75ba22a-ed6a-41f0-830d-ba153dc3f15e' source='A' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=232) metadata={} created_at=datetime.datetime(2025, 9, 15, 9, 16, 14, 223507, tzinfo=datetime.timezone.utc) content="Once upon a time, in a cozy seaside village, there lived a curious cat named Oliver. With silky gray fur and bright green eyes, he loved to explore every nook and cranny of his surroundings. Every morning, Oliver would perch on the windowsill, watching the waves dance against the shore, dreaming of adventures beyond the village.\n\nOne sunny afternoon, Oliver spotted a colorful butterfly fluttering near the garden. Entranced, he tiptoed after it, his paws barely making a sound on the stone path. The butterfly led him to a h

In [ ]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main():
    # Initialize agents with OpenAI model clients.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="Detect if the input is in Chinese. If it is, say 'yes', else say 'no', and nothing else.",
    )
    agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input to English.")
    agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Chinese.")

    # Create a directed graph with conditional branching flow A -> B ("yes"), A -> C (otherwise).
    builder = DiGraphBuilder()
    builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    # Create conditions as callables that check the message content.
    builder.add_edge(agent_a, agent_b, condition=lambda msg: "yes" in msg.to_model_text())
    builder.add_edge(agent_a, agent_c, condition=lambda msg: "yes" not in msg.to_model_text())
    graph = builder.build()

    # Create a GraphFlow team with the directed graph.
    team = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

    # Run the team and print the events.
    async for event in team.run_stream(task="AutoGen is a framework for building AI agents."):
        print(event)


asyncio.run(main())